In [9]:
from openai import OpenAI
import random

with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

client = OpenAI(api_key=api_key)

sys_instruct="All prompts should be answered with an in depth paper with an introduction, middle and end structured into chapters that is about 4 pages, written in markdown and include sources. Dont answer anything with less than 4 pages. Dont write anything other than the paper."
topics = ["technology", "science", "history", "art", "literature", "politics", "economics", "philosophy"]

for _ in range(30):
    topic = random.choice(topics)
    completion = client.chat.completions.create(
    model="gpt-4o",
    store=True,
    messages=[
        {"role": "developer", "content": sys_instruct},
        {"role": "user", "content": f"Write a paper about {topic}. The specific topic is up to you."}
    ]
    )


    try: 
        with open(f"outputs/{_}_{topic}.md", "a") as f:
            f.write(completion.choices[0].message.content)
    except:
        pass